# 风险管理作业7
-----
2501210025 叶云鹏

### (一）CDO产品的定价方法整理



#### 1. **损失过程定义**
总损失 L(t) 表示到时点 t 的资产池累计损失：
$$
L(t) = \sum_{i=1}^n L_i I_{\{\tau_i \leq t\}}
$$
- $n$：资产数量。
- $L_i$：第 i 个资产的损失给定违约（LGD，通常为 1 - 回收率）。
- $τ_i$：第 i 个资产的违约时间（随机变量，常服从指数分布强度 λ_i）。
- $I_{\{·\}}$：指示函数，若 τ_i ≤ t 则为1（已违约）。捕捉组合信用风险

#### 2. **违约腿（Default Leg）：预期违约支付**
违约腿计算保护卖方（CDO 发行方）在违约时需支付的预期补偿，
- **连续形式**：
$$
E^P \left[ \int_0^T B(0,t) \, dM_t \right] = B(0,T) E^P [M(T)] + \int_0^T f(0,t) B(0,t) E^P [M(t)] \, dt
$$

- **离散形式**：
$$
\sum_{i=1}^n B(0, t_i) [M(t_i) - M(t_{i-1})]
$$

#### 3. **保费腿（Premium Leg）：预期保费现金流**
保费腿计算保护买方支付的定期保费现值，依赖于未实现损失（只有在损失未达脱离点 D 时支付）。
- **离散形式**：
$$
E^P \left[ \sum_{i=1}^M \Delta_{i-1,i} W B(0, t_i) [D - C] I_{\{L(t_i) \leq C\}} + \sum_{i=1}^M \Delta_{i-1,i} W B(0, t_i) [D - L(t_i)] I_{\{C \leq L(t_i) \leq D\}} \right]
$$
 
- **连续形式**：
$$
W E^P \left[ \int_0^T B(0,t) g(L(t)) \, dt \right]
$$
  其中 $g(L(t)) = min\{ max\{D - L(t), 0\}, D - C \}$

#### 4. **公平保费（Fair Spread）计算**
在无套利条件下，违约腿 = 保费腿，解得瞬时公平保费 W：
$$
W = \frac{E^P \left[ \int_0^T B(0,t) \, dM_t \right]}{E^P \left[ \int_0^T B(0,t) g(L(t)) \, dt \right]}
$$


### (二）计算债券组合期望与方差
考虑两只面值为100的债券：
+ （1）债券1:BBB级，高级无担保，6%年息票率，五年期；
+ （2）债券2: A级，高级无担保，5%年票息率，三年期

假设两个债券对应的公司的资产收益率都服从正态分布，二者的相关系数为0.5。
利用转移矩阵，收益率曲线表，回收率表计算在一年末债券组合价值的期望及方差

转移表

|**初始评级**|**AAA**|**AA**|**A**|**BBB**|**BB**|**B**|**CCC**|**Default (违约)**|
|---|---|---|---|---|---|---|---|---|
|**AAA**|90.81|8.33|0.68|0.06|0.12|0|0|0|
|**AA**|0.70|90.65|7.79|0.64|0.06|0.14|0.02|0|
|**A**|0.09|2.27|91.05|5.52|0.74|0.26|0.01|0.06|
|**BBB**|0.02|0.33|5.95|86.93|5.30|1.17|0.12|0.18|
|**BB**|0.03|0.14|0.67|7.73|80.53|8.84|1.00|1.06|
|**B**|0|0.11|0.24|0.43|6.48|83.46|4.07|5.20|
|**CCC**|0.22|0|0.22|1.30|2.38|11.24|64.86|19.79|

收益率曲线表
| **评级类别 (Category)** | **第 1 年 (Year 1)** | **第 2 年 (Year 2)** | **第 3 年 (Year 3)** | **第 4 年 (Year 4)** |
| ------------------- | ------------------ | ------------------ | ------------------ | ------------------ |
| **AAA**             | 3.60               | 4.17               | 4.73               | 5.12               |
| **AA**              | 3.65               | 4.22               | 4.78               | 5.17               |
| **A**               | 3.72               | 4.32               | 4.93               | 5.32               |
| **BBB**             | 4.10               | 4.67               | 5.25               | 5.63               |
| **BB**              | 5.55               | 6.02               | 6.78               | 7.27               |
| **B**               | 6.05               | 7.02               | 8.03               | 8.52               |
| **CCC**             | 15.05              | 15.02              | 14.03              | 13.52              |
回收率表

|**偿付等级 (Seniority Class)**|**平均回收率 (Mean) (%)**|**标准差 (Standard Deviation) (%)**|
|---|---|---|
|**有担保的高级债 (Senior Secured)**|53.80|26.86|
|**无担保的高级债 (Senior Unsecured)**|51.13|25.45|
|**次级高级债 (Senior Subordinated)**|38.52|23.81|
|**次级债 (Subordinated)**|32.74|20.18|
|**初级次级债 (Junior Subordinated)**|17.09|10.90|

In [1]:
import numpy as np
from scipy.stats import norm, multivariate_normal

rho = 0.5
recov_mean = 51.13
recov_sd = 25.45
recov_var = recov_sd ** 2

yields_dict = {
'AAA': [3.60, 4.17, 4.73, 5.12],
'AA': [3.65, 4.22, 4.78, 5.17],
'A': [3.72, 4.32, 4.93, 5.32],
'BBB': [4.10, 4.67, 5.25, 5.63],
'BB': [5.55, 6.02, 6.78, 7.27],
'B': [6.05, 7.02, 8.03, 8.52],
'CCC': [15.05, 15.02, 14.03, 13.52]
}

def compute_pv(rating, maturity, coupon):
    ys = np.array(yields_dict[rating]) / 100.0
    pv = 0.0
    for t in range(1, maturity + 1):
        cf = coupon if t < maturity else coupon + 100
        pv += cf / (1 + ys[t-1]) ** t
    return pv

val1 = {}
for r in yields_dict:
    pv = compute_pv(r, 4, 6)
    val1[r] = 6 + pv
val1['D'] = recov_mean

val2 = {}
for r in yields_dict:
    pv = compute_pv(r, 2, 5)
    val2[r] = 5 + pv
val2['D'] = recov_mean

ratings = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'D']
probs1 = np.array([0.0002, 0.0033, 0.0595, 0.8693, 0.0530, 0.0117, 0.0012, 0.0018])
probs2 = np.array([0.0009, 0.0227, 0.9105, 0.0552, 0.0074, 0.0026, 0.0001, 0.0006])

E_U1 = np.sum(probs1 * [val1[r] for r in ratings])
E_U1_sq = np.sum(probs1 * [val1[r]**2 for r in ratings])
Var_U1 = E_U1_sq - E_U1**2

E_U2 = np.sum(probs2 * [val2[r] for r in ratings])
E_U2_sq = np.sum(probs2 * [val2[r]**2 for r in ratings])
Var_U2 = E_U2_sq - E_U2**2

P_def1 = probs1[-1]
P_def2 = probs2[-1]

ordered_ratings = ['D', 'CCC', 'B', 'BB', 'BBB', 'A', 'AA', 'AAA']
val1_ordered = [val1[r] for r in ordered_ratings]
val2_ordered = [val2[r] for r in ordered_ratings]

probs_worst_to_best1 = [probs1[7], probs1[6], probs1[5], probs1[4], probs1[3], probs1[2], probs1[1], probs1[0]]
cum_probs1 = np.cumsum(probs_worst_to_best1)
thresholds1 = norm.ppf(cum_probs1[:-1])
thresh_full1 = np.concatenate(([-np.inf], thresholds1, [np.inf]))

probs_worst_to_best2 = [probs2[7], probs2[6], probs2[5], probs2[4], probs2[3], probs2[2], probs2[1], probs2[0]]
cum_probs2 = np.cumsum(probs_worst_to_best2)
thresholds2 = norm.ppf(cum_probs2[:-1])
thresh_full2 = np.concatenate(([-np.inf], thresholds2, [np.inf]))

def biv_cdf(x, y, rho=0.5):
    if np.isinf(x) and x < 0 or np.isinf(y) and y < 0:
        return 0.0
    if np.isinf(x) and x > 0:
        if np.isinf(y) and y > 0:
            return 1.0
        else:
            return norm.cdf(y)
    if np.isinf(y) and y > 0:
        return norm.cdf(x)
    return multivariate_normal(mean=[0,0], cov=[[1, rho], [rho, 1]]).cdf([x, y])

def prob_rect(low1, up1, low2, up2):
    return biv_cdf(up1, up2) - biv_cdf(up1, low2) - biv_cdf(low1, up2) + biv_cdf(low1, low2)

E_U1_U2 = 0.0
for i in range(8):
    low1 = thresh_full1[i]
    up1 = thresh_full1[i+1]
    v1 = val1_ordered[i]
    for j in range(8):
        low2 = thresh_full2[j]
        up2 = thresh_full2[j+1]
        v2 = val2_ordered[j]
        p = prob_rect(low1, up1, low2, up2)
        E_U1_U2 += p * v1 * v2

cov_U = E_U1_U2 - E_U1 * E_U2
expect = E_U1 + E_U2
var_U = Var_U1 + Var_U2 + 2 * cov_U
var_total = var_U + P_def1 * recov_var + P_def2 * recov_var

print('债券组合价值的期望',expect)

print('债券组合价值的方差',var_total)

债券组合价值的期望 213.27082474478436
债券组合价值的方差 13.583873578155364
